In [ ]:
import transformers
import torch
import sqlite3
import re #regex
import ast #abstract syntax trees

In [ ]:
local_path = r'C:\Users\MTGro\Desktop\llm_models\Mistral-7B-Instruct-v0.1'

device = 'cuda'

tokenizer = transformers.AutoTokenizer.from_pretrained(local_path)

model = transformers.AutoModelForCausalLM.from_pretrained(local_path
                                                          , local_files_only = True
                                                          ,  device_map=device
                                                          , torch_dtype=torch.float16)

In [ ]:
user_content= '''
You are a creative writer working on a comedy novel.

In the style of Oscar Wilde, write the first paragraph of
a story where a guy accidentally meets the love of his life.

'''

In [ ]:
messages = [{"user", "content": user_content}]

In [ ]:
model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)

decoded = tokenizer.batch_decode(generated_ids)

print(decoded[0])

In [ ]:
def extract_text_after_inst(text):
    # Use regular expression to find text after [/INST]
    match = re.search(r'\[/INST\](.*)', text, re.DOTALL)
    
    if match:
        # Extract and return the text after [/INST]
        result = match.group(1).strip()
        return result
    else:
        # Return None if [/INST] is not found
        return None

output =  extract_text_after_inst(decoded[0]).replace('</s>', '')

In [ ]:
conn = sqlite3.connect('story_prompts2.sqlite')
c = conn.cursor()
create_table = '''
CREATE TABLE "chat" (
 "ID" INTEGER NOT NULL UNIQUE,
 "prompt" TEXT NOT NULL,
 "output" TEXT,
 PRIMARY KEY("ID" AUTOINCREMENT)
);

'''
c.execute(create_table)
conn.commit()
conn.close()

In [ ]:
def chat(user_content):
    conn = sqlite3.connect('story_prompts2.sqlite')
    c = conn.cursor()
    messages = [
        {"role": "user", "content":user_content}
  
    ]

    model_inputs= tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)
     
    generated_ids = model.generate(model_inputs, max_new_tokens=4000, do_sample=False)
    decoded = tokenizer.batch_decode(generated_ids)
    output =  extract_text_after_inst(decoded[0]).replace('</s>', '').replace("'", "''")
    #print(output)
    user_content = user_content.replace("'", "''")
    c.execute( 
      f'''INSERT INTO chat (prompt, output) VALUES ('{user_content}', '{output}')''') 
    conn.commit()
    conn.close()
    #print(decoded[0])
    return print(extract_text_after_inst(decoded[0]))